In [1]:
#載入
import sqlite3
import pandas as pd
import numpy as np
import datetime
import time
from os import listdir
from os.path import isfile, isdir, join
import os
import talib as tb
import talib.abstract as tbab
import math #判斷nan值用
import csv

conn = sqlite3.connect('TSE.db')
c = conn.cursor()

In [132]:
#條件: 昨日法人5日買超小於100億，當天股價大於前月最高價
#前月最高 當天上漲不能超過50點 當天法人買超100億以上 隔日早上需賣出 若隔日下跌不足0.5%可續留
def price_max(date):
    return pd.read_sql_query('SELECT MAX("收盤指數") FROM(SELECT * FROM "發行量加權股價指數" where date<='+str(date)+'  ORDER BY date DESC  Limit 20);', conn).iloc[0][0]

date_ary =  pd.read_sql_query('SELECT * FROM "發行量加權股價指數" where date>=20180122;', conn)['date'].tolist()
for i in date_ary:
    target_date = str(i)
    data = pd.read_sql_query('SELECT * FROM "發行量加權股價指數"  where date < '+target_date+' ORDER BY date DESC Limit 5;', conn)
    five_sum = round(sum(data['合計'].tolist())/100000000,2)
    if five_sum<100:
        target_point =  pd.read_sql_query('SELECT "收盤指數" FROM "發行量加權股價指數"  where date = '+str(i)+';', conn).iloc[0][0]
        sum_data = pd.read_sql_query('SELECT * FROM "發行量加權股價指數"  where date = '+str(i)+';', conn)['合計'][0]
        grow_point = pd.read_sql_query('SELECT * FROM "發行量加權股價指數"  where date = '+str(i)+';', conn)['漲跌百分比(%)'][0]
        if price_max(i) <=target_point and sum_data< 10000000000 and grow_point<0.5:
            print(i)

20180528
20180801
20190425
20190611
20190612


In [139]:
data = pd.read_sql_query('SELECT * FROM "發行量加權股價指數" where date >= 20190425 limit 5;', conn)
data

,date,收盤指數,漲跌(+/-),漲跌點數,漲跌百分比(%),自營商(自行買賣),自營商(避險),投信,外資及陸資(不含外資自營商),外資自營商,合計
0,20190425,11039.86,+,12.22,0.11,-35502558,234886362,-197554868,755910589,2192360,757739525
1,20190426,10952.47,-,87.39,-0.79,-481244092,-191449787,-678414454,-6476339287,-1793640,-7827447620
2,20190429,10939.06,-,13.41,-0.12,-448112687,263080717,-1328007872,-384336006,484680,-1897375848
3,20190430,10967.73,+,28.67,0.26,248481971,233676524,16586189,104700093,-3004440,603444777
4,20190502,11004.49,+,36.76,0.34,214383917,72090924,-42868930,5296485685,-700210,5540091596


In [117]:
data = pd.read_sql_query('SELECT "自營商(自行買賣)"+"自營商(避險)" as"自營商" FROM "發行量加權股價指數"  ORDER BY date DESC Limit 5;', conn)
data

,自營商
0,433991203
1,875087433
2,-224829440
3,3882661638
4,-715203288


In [64]:
data = pd.read_sql_query('SELECT date,"收盤指數" ,"合計","自營商(自行買賣)"+"自營商(避險)" as"自營商","投信","外資及陸資(不含外資自營商)"+"外資自營商" as "外資" FROM "發行量加權股價指數"  ORDER BY date DESC Limit 20;', conn)
data

,date,收盤指數,合計,自營商,投信,外資
0,20190715,10876.43,2030525842,433991203,830997378,763198751
1,20190712,10824.35,837857647,875087433,536460241,-569818967
2,20190711,10843.42,5294580253,-224829440,654797490,4858960693
3,20190710,10798.48,5216006631,3882661638,148889688,1183026865
4,20190709,10702.78,-4420733662,-715203288,-480663487,-3220999107
5,20190708,10751.22,-8311779475,-277674161,-635677702,-7398546842
6,20190705,10785.73,1065325685,1173905805,-318385452,211249792
7,20190704,10775.90,-910379697,-642082427,-412163806,140009676
8,20190703,10743.77,-9662297637,-560511919,-1072694800,-8026426478
9,20190702,10865.12,-3233134129,1352153892,379616780,-4957629841


In [100]:
11724107233/100000000

117.24107233

,date,收盤指數,漲跌(+/-),漲跌點數,漲跌百分比(%),自營商(自行買賣),自營商(避險),投信,外資及陸資(不含外資自營商),外資自營商,合計
0,20180102,10710.73,+,67.87,0.64,885262708,2092268647,-74245611,8648245117,0,11551530861
1,20180103,10801.57,+,90.84,0.85,1679725617,1506486418,999578570,3725762356,0,7911552961
2,20180104,10848.63,+,47.06,0.44,1020028250,817845859,-141653974,5474260013,0,7170480148
3,20180105,10879.80,+,31.17,0.29,947461930,688283515,-334919686,3480267389,0,4781093148
4,20180108,10915.75,+,35.95,0.33,518929950,-532258916,934560469,5009518003,0,5930749506
